# PGM - 1. deo

## Osnovni pojmovi
- $X$ - slučajna promenljiva (broj dobijen na gornjoj strani kockice)
- $\{1, 2, 3, 4, 5, 6\}$ - skup vrednosti slučajne promenljive
- $P(X=1)$ - Verovatnoća da slučajna promenljiva $X$ ima vrednost $1$
- $P(X) = \{1: 1/6, 2: 1/6, ... , 6: 1/6 \}$ - Funkcija raspodele vrednosti slučajne promenljive $X$
- $P(X=1, Y=2)$ - Zajednička verovatnoća da promenljive $X$ i $Y$ istovremeno imaju vrednosti $1$ i $2$
- $P(X, Y)$ - Zajednička raspodela verovatnoća vrednosti $X$ i $Y$
- $P(X=1|Y=2)$ - Uslovna verovatnoća, verovatnoća da promenljiva $X$ ima vrednost $1$ AKO promenljiva $Y$ ima vrednost $2$
- $P(X|Y)$ - Uslovna raspodela verovatnoća vrednosti promenljive $X$ u odnosu na vrednosti promenljive $Y$

## Osnovne jednakosti
- $P(X,Y) = P(X|Y) \cdot P(Y) = P(Y|X) \cdot P(X)$
- $P(X, Y, Z) = P(X|Y, Z) \cdot P(Y|Z) \cdot P(Z)$ - Lančano pravilo 
- Ako su $X$ i $Y$ NEZAVISNE promenljive: $P(X|Y) = P(X), P(Y|X) = P(Y)$
- $P(X|Y) = \frac{P(Y|X) * P(X)}{P(Y)}$ - Bajesova teorema

In [8]:
# Bajesovski model
from pgmpy.models import BayesianModel

# Conditional Probability Table, funkcije uslovne raspodele tabelarno predstavljena
from pgmpy.factors.discrete import TabularCPD 

# Algoritam koji vrši zaključivanje 
from pgmpy.inference import VariableElimination

ImportError: cannot import name 'BayesianNetwork' from 'pgmpy.models' (/usr/local/lib/python3.8/site-packages/pgmpy/models/__init__.py)

In [9]:
# Kasalj, Temperatura, X-ray, COVID

'''
kasalj   temperatura
     \   /
     X-ray
       |
     covid
'''         

K_cpd = TabularCPD(
    variable="kasalj",
    variable_card=2,
    values=[[0.4],[0.6]],
    state_names = { 'kasalj': ['NE', 'DA'] }
)

T_cpd = TabularCPD(
    variable="temperatura",
    variable_card=2,
    values=[[0.3],[0.7]],
    state_names = { 'temperatura': ['N', 'V'] }
)

X_cpd = TabularCPD(
    variable="X-ray",
    variable_card=2,
    evidence=['kasalj','temperatura'],
    evidence_card=[2,2],
    values=[
        [0.1,0.2,0.3,0.4],
        [0.9,0.8,0.7,0.6],
    ],
    state_names = {
        'kasalj': ['NE', 'DA'],
        'temperatura': ['N', 'V'],
        'X-ray': ['LOS', 'DOBAR']
    }
)

C_cpd = TabularCPD(
    variable="covid",
    variable_card=2,
    evidence=['X-ray'],
    evidence_card=[2],
    values=[
        [0.2,0.7],
        [0.8,0.3],
    ],
    state_names = {
        'X-ray': ['LOS', 'DOBAR'],
        'covid': ['NE', 'DA']
    }
)

In [11]:
# Konstrukcija modela pomocu CPD i veza izmedju cvorova
model = BayesianModel([('kasalj', 'X-ray'), ('temperatura', 'X-ray'), ('X-ray', 'covid')])
model.add_cpds(K_cpd, T_cpd, X_cpd, C_cpd)

In [12]:
# Provera ispravnosti modela (probati sa izmenama vrednosti CPD)
model.check_model()

True

In [13]:
# Izvršavanje probabilističkih upita
inference = VariableElimination(model)
res = inference.query(['covid'])
print(res)

Eliminating: temperatura: 100%|██████████| 3/3 [00:00<00:00, 350.26it/s]

+-----------+--------------+
| covid     |   phi(covid) |
+===========+==============+
| covid(NE) |       0.5550 |
+-----------+--------------+
| covid(DA) |       0.4450 |
+-----------+--------------+


In [19]:
# Izvršavanje upita uz opažanje pojedinih vrednosti 
inference = VariableElimination(model)
res = inference.query(['covid'], evidence={'kasalj': 'DA', 'temperatura': 'V'})
print(res)

Eliminating: X-ray: 100%|██████████| 1/1 [00:00<00:00, 163.85it/s]

+-----------+--------------+
| covid     |   phi(covid) |
+===========+==============+
| covid(NE) |       0.5000 |
+-----------+--------------+
| covid(DA) |       0.5000 |
+-----------+--------------+


In [17]:
# Primer nezavisnosti usled zatvaranja toka zavisnosti u grafu
inference = VariableElimination(model)
res1 = inference.query(['covid'], evidence={'X-ray': 'LOS'})
print(res1)
res2 = inference.query(['covid'], evidence={'X-ray': 'LOS', 'kasalj': 'NE'})
print(res2)

Eliminating: temperatura: 100%|██████████| 1/1 [00:00<00:00, 65.11it/s]

+-----------+--------------+
| covid     |   phi(covid) |
+===========+==============+
| covid(NE) |       0.2000 |
+-----------+--------------+
| covid(DA) |       0.8000 |
+-----------+--------------+
+-----------+--------------+
| covid     |   phi(covid) |
+===========+==============+
| covid(NE) |       0.2000 |
+-----------+--------------+
| covid(DA) |       0.8000 |
+-----------+--------------+


In [20]:
# Primer MAP upita
inference = VariableElimination(model)
res = inference.map_query(['covid'], evidence={'X-ray': 'LOS', 'kasalj': 'NE'})
print(res)

Eliminating: temperatura: 100%|██████████| 1/1 [00:00<00:00, 324.06it/s]

{'covid': 'DA'}


In [21]:
# Pronađene nezavisnosti u grafu
model.get_independencies()

(kasalj _|_ temperatura)
(kasalj _|_ covid | X-ray)
(kasalj _|_ covid | X-ray, temperatura)
(temperatura _|_ kasalj)
(temperatura _|_ covid | X-ray)
(temperatura _|_ covid | X-ray, kasalj)
(covid _|_ kasalj, temperatura | X-ray)
(covid _|_ temperatura | X-ray, kasalj)
(covid _|_ kasalj | X-ray, temperatura)

In [22]:
# Pronađene lokalne nezavisnosti vezane za promenljivu "kasalj"
model.local_independencies('kasalj')

(kasalj _|_ temperatura)

In [26]:
# Pronađene aktivne putanje zavisnosti u odnosu na čvor "kasalj"
model.active_trail_nodes('kasalj')

{'kasalj': {'X-ray', 'covid', 'kasalj'}}

In [28]:
# Pronađene aktivne putanje zavisnosti u odnosu na čvor "kasalj"
# pri opažanju vrednosti promenljive X-ray (zatvaranje V-strukture u čvoru X-ray)
model.active_trail_nodes('kasalj', observed=['X-ray'])

{'kasalj': {'kasalj', 'temperatura'}}